In [12]:
# Importaciones de librerias y src
import sys
import os
sys.path.append("../")
from src import soporte_codigos as sop
# from src import llamar_api_demanda as demanda
# from src import llamar_api_generacion as generacion


In [13]:
import sys
import os

import json
import pandas as pd 

import requests

In [14]:
headers={"Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es"}
ruta_demanda = r"C:\\Users\\Administrador\\Desktop\\RCM\\laboratorio-modulo5-leccion01-elt-extraccion\\data\\demanda"
os.makedirs(ruta_demanda, exist_ok=True)

In [15]:
lista_anios=[2019,2020,2021]

In [16]:
for anio in lista_anios:
    for nombre, ccaa in sop.codigos_comunidades.items():
        
        url_demanda=f"https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={anio}-01-01T00:00&end_date={anio}-12-31T23:59&time_trunc=month&geo_trunk=electric_system&geo_limit=ccaa&geo_ids={ccaa}"

        response = requests.get(url_demanda, headers=headers)
        responsa= response.json()  
        df_demanda= pd.DataFrame(responsa['included'][0]['attributes']['values']) 
        df_demanda["cod_ccaa"]= ccaa
        df_demanda.to_csv(os.path.join(ruta_demanda, f'{nombre}_{anio}.csv'))

In [19]:
ruta_generacion = r"C:\\Users\\Administrador\\Desktop\\RCM\\laboratorio-modulo5-leccion01-elt-extraccion\\data\\generacion"
os.makedirs(ruta_generacion, exist_ok=True)

for anio in lista_anios:
    for nombre, ccaa in sop.codigos_comunidades.items():
        
        url_generacion=f"https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={anio}-01-01T00:00&end_date={anio}-12-31T23:59&time_trunc=month&geo_trunk=electric_system&geo_limit=ccaa&geo_ids={ccaa}"

        response = requests.get(url_generacion, headers=headers)
        if response.status_code != 200:
            continue
        else:
            responsa= response.json()  
            df_completo= pd.DataFrame()
            for i in range(0, len(responsa['included'])):
                df_tipo=pd.DataFrame(responsa['included'][i]['attributes']['values'])
                df_tipo["type"]= responsa["included"][i]["type"]
                df_tipo["cod_ccaa"]= ccaa
                df_completo= pd.concat([df_completo, df_tipo ])
            df_oferta= df_completo.to_csv(os.path.join(ruta_generacion, f'{nombre}_{anio}.csv'))